In [2]:
import numpy as np
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import cosine_similarity


import warnings
warnings.filterwarnings("ignore")

books=pd.read_csv('Dataset/SampleBooks.csv',header=[0])
users=pd.read_csv('Dataset/SampleUsers.csv',header=[0])
ratings=pd.read_csv('Dataset/SampleRatings.csv',header=[0])
#books.info()
users.drop(['Location', 'Age'], axis=1, inplace=True)
#users.info()
ratings.info()
#book_data.describe()
#ratings_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   User-ID      12 non-null     int64
 1   ISBN         12 non-null     int64
 2   Book-Rating  12 non-null     int64
dtypes: int64(3)
memory usage: 416.0 bytes


In [3]:
print(books.iloc[0]['ISBN'])

12345679


In [4]:
print(books.head())

       ISBN                                Book-Title
0  12345679      Harry Potter and the Goblet of Fire 
1  12345678                                      ABCD
2  12345677  Harry Potter and the Prisoner of Azkaban
3  12345676                                       FGH
4  12345675     Harry Potter and the Sorcerer's Stone


In [5]:
dataset1 = pd.merge(books, ratings, on='ISBN', how='inner')
#dataset = pd.merge(dataset, users, on='User-ID', how='inner')
dataset1.info()
print(dataset1)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         12 non-null     int64 
 1   Book-Title   12 non-null     object
 2   User-ID      12 non-null     int64 
 3   Book-Rating  12 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 480.0+ bytes
        ISBN                                Book-Title  User-ID  Book-Rating
0   12345679      Harry Potter and the Goblet of Fire    276725            9
1   12345679      Harry Potter and the Goblet of Fire    276726            7
2   12345679      Harry Potter and the Goblet of Fire    276733           10
3   12345678                                      ABCD   276727            6
4   12345678                                      ABCD   276737            9
5   12345677  Harry Potter and the Prisoner of Azkaban   276729            8
6   12345676                                       FGH   2

In [6]:
dataset1 = pd.merge(dataset1, users, on='User-ID', how='inner')
print(dataset1)


        ISBN                                Book-Title  User-ID  Book-Rating
0   12345679      Harry Potter and the Goblet of Fire    276725            9
1   12345670                                       OPQ   276725            9
2   12345679      Harry Potter and the Goblet of Fire    276726            7
3   12345679      Harry Potter and the Goblet of Fire    276733           10
4   12345678                                      ABCD   276727            6
5   12345678                                      ABCD   276737            9
6   12345677  Harry Potter and the Prisoner of Azkaban   276729            8
7   12345675     Harry Potter and the Sorcerer's Stone   276736            9
8   12345673   Harry Potter and the Chamber of Secrets   276744            8
9   12345672                                    ABCDEF   276745            7
10  12345671                                   ABCDEFG   276746            6


In [7]:
df = pd.DataFrame(dataset1['Book-Title'].value_counts())
print(df)

                                          Book-Title
Harry Potter and the Goblet of Fire                3
ABCD                                               2
OPQ                                                1
Harry Potter and the Prisoner of Azkaban           1
Harry Potter and the Sorcerer's Stone              1
Harry Potter and the Chamber of Secrets            1
ABCDEF                                             1
ABCDEFG                                            1


In [8]:
df = pd.DataFrame(dataset1['Book-Title'].value_counts())
df['Total-Ratings'] = df['Book-Title']
df['Book-Title'] = df.index
df.reset_index(level=0, inplace=True)
df = df.drop('index',axis=1)
df = dataset1.merge(df, left_on = 'Book-Title', right_on = 'Book-Title', how = 'left')
print(df.info())
print(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ISBN           11 non-null     int64 
 1   Book-Title     11 non-null     object
 2   User-ID        11 non-null     int64 
 3   Book-Rating    11 non-null     int64 
 4   Total-Ratings  11 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 528.0+ bytes
None
        ISBN                                Book-Title  User-ID  Book-Rating  \
0   12345679      Harry Potter and the Goblet of Fire    276725            9   
1   12345670                                       OPQ   276725            9   
2   12345679      Harry Potter and the Goblet of Fire    276726            7   
3   12345679      Harry Potter and the Goblet of Fire    276733           10   
4   12345678                                      ABCD   276727            6   
5   12345678                                      ABCD   

In [9]:
popularity_threshold = 1
popular_book = df[df['Total-Ratings'] >= popularity_threshold]
popular_book = popular_book.reset_index(drop = True)
print(popular_book)

        ISBN                                Book-Title  User-ID  Book-Rating  \
0   12345679      Harry Potter and the Goblet of Fire    276725            9   
1   12345670                                       OPQ   276725            9   
2   12345679      Harry Potter and the Goblet of Fire    276726            7   
3   12345679      Harry Potter and the Goblet of Fire    276733           10   
4   12345678                                      ABCD   276727            6   
5   12345678                                      ABCD   276737            9   
6   12345677  Harry Potter and the Prisoner of Azkaban   276729            8   
7   12345675     Harry Potter and the Sorcerer's Stone   276736            9   
8   12345673   Harry Potter and the Chamber of Secrets   276744            8   
9   12345672                                    ABCDEF   276745            7   
10  12345671                                   ABCDEFG   276746            6   

    Total-Ratings  
0               3  

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df = 1, stop_words='english')
tfidf_matrix = tf.fit_transform(popular_book['Book-Title'])
tfidf_matrix.shape

normalized_df = tfidf_matrix.astype(np.float64)
cosine_similarities = cosine_similarity(normalized_df, normalized_df)
cosine_similarities.shape

(11, 11)

In [77]:
print(cosine_similarities)

[[1.         0.         1.         1.         0.         0.
  0.28788276 0.28788276 0.28788276 0.         0.        ]
 [0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.        ]
 [1.         0.         1.         1.         0.         0.
  0.28788276 0.28788276 0.28788276 0.         0.        ]
 [1.         0.         1.         1.         0.         0.
  0.28788276 0.28788276 0.28788276 0.         0.        ]
 [0.         0.         0.         0.         1.         1.
  0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         1.         1.
  0.         0.         0.         0.         0.        ]
 [0.28788276 0.         0.28788276 0.28788276 0.         0.
  1.         0.18561404 0.18561404 0.         0.        ]
 [0.28788276 0.         0.28788276 0.28788276 0.         0.
  0.18561404 1.         0.18561404 0.         0.        ]
 [0.28788276 0.         0.28788276 0.28788276 0.        

In [65]:
print("Recommended Books:\n")
isbn = books.loc[books['Book-Title'] == 'Harry Potter and the Prisoner of Azkaban'].reset_index(drop = True).iloc[0]['ISBN']
content = []

idx = popular_book.index[popular_book['ISBN'] == isbn].tolist()[0]
similar_indices = cosine_similarities[idx].argsort()[::-1]
similar_items = []
for i in similar_indices:
    if popular_book['Book-Title'][i] != 'Harry Potter and the Prisoner of Azkaban' and popular_book['Book-Title'][i] not in similar_items and len(similar_items) < 4:
        similar_items.append(popular_book['Book-Title'][i])
        content.append(popular_book['Book-Title'][i])

for book in similar_items:
    print(book)

Recommended Books:

Harry Potter and the Goblet of Fire 
Harry Potter and the Order of the Phoenix
Harry Potter and the Chamber of Secrets
Harry Potter and the Sorcerer's Stone


In [78]:
print("Recommended Books:\n")
isbn = books.loc[books['Book-Title'] == 'ABCD'].reset_index(drop = True).iloc[0]['ISBN']
content = []

idx = popular_book.index[popular_book['ISBN'] == isbn].tolist()[0]
similar_indices = cosine_similarities[idx].argsort()[::-1]
similar_items = []
for i in similar_indices:
    if popular_book['Book-Title'][i] != 'ABCD' and popular_book['Book-Title'][i] not in similar_items and len(similar_items) < 2:
        similar_items.append(popular_book['Book-Title'][i])
        content.append(popular_book['Book-Title'][i])

for book in similar_items:
    print(book)

Recommended Books:

ABCDEFG
ABCDEF


In [23]:
from scipy.sparse import csr_matrix
from pandas.api.types import is_numeric_dtype
from sklearn.neighbors import NearestNeighbors

data = (dataset1.groupby(by = ['Book-Title'])['Book-Rating'].count().reset_index().
        rename(columns = {'Book-Rating': 'Total-Rating'})[['Book-Title', 'Total-Rating']])

result = pd.merge(data, dataset1, on='Book-Title',)
result = result[result['Total-Rating'] >= popularity_threshold]
result = result.reset_index(drop = True)

matrix = result.pivot_table(index = 'Book-Title', columns = 'User-ID', values = 'Book-Rating').fillna(0)
up_matrix = csr_matrix(matrix)

model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(up_matrix)

bookName="ABCD"
distances, indices = model.kneighbors(matrix.loc[bookName].values.reshape(1, -1), n_neighbors = 4+1)
print("\nRecommended books:\n")
for i in range(0, len(distances.flatten())):
    if i > 0:
        print(matrix.index[indices.flatten()[i]])


Recommended books:

Harry Potter and the Prisoner of Azkaban
Harry Potter and the Sorcerer's Stone
Harry Potter and the Chamber of Secrets
Harry Potter and the Goblet of Fire 


In [24]:
print(matrix.loc['ABCD'])

User-ID
276725    0.0
276726    0.0
276727    6.0
276729    0.0
276733    0.0
276736    0.0
276737    9.0
276744    0.0
276745    0.0
276746    0.0
Name: ABCD, dtype: float64


In [14]:
print(distances,indices)

[[2.22044605e-16 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00]] [[0 5 6 3 4]]


In [15]:
print(distances.flatten())

[2.22044605e-16 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00]


In [11]:
print(data)

                                 Book-Title  Total-Rating
0                                      ABCD             2
1                                    ABCDEF             1
2                                   ABCDEFG             1
3   Harry Potter and the Chamber of Secrets             1
4      Harry Potter and the Goblet of Fire              3
5  Harry Potter and the Prisoner of Azkaban             1
6     Harry Potter and the Sorcerer's Stone             1
7                                       OPQ             1


In [90]:
print(matrix)

User-ID                                   276725  276726  276727  276729  \
Book-Title                                                                 
ABCD                                         0.0     0.0     6.0     0.0   
ABCDEF                                       0.0     0.0     0.0     0.0   
ABCDEFG                                      0.0     0.0     0.0     0.0   
Harry Potter and the Chamber of Secrets      0.0     0.0     0.0     0.0   
Harry Potter and the Goblet of Fire          9.0     7.0     0.0     0.0   
Harry Potter and the Prisoner of Azkaban     0.0     0.0     0.0     8.0   
Harry Potter and the Sorcerer's Stone        0.0     0.0     0.0     0.0   
OPQ                                          9.0     0.0     0.0     0.0   

User-ID                                   276733  276736  276737  276744  \
Book-Title                                                                 
ABCD                                         0.0     0.0     9.0     0.0   
ABCDEF     

In [91]:
print(up_matrix)
print(distances,indices)

  (0, 2)	6.0
  (0, 6)	9.0
  (1, 8)	7.0
  (2, 9)	6.0
  (3, 7)	8.0
  (4, 0)	9.0
  (4, 1)	7.0
  (4, 4)	10.0
  (5, 3)	8.0
  (6, 5)	9.0
  (7, 0)	9.0
[[2.22044605e-16 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00]] [[0 5 6 3 4]]


In [43]:
testdf = pd.DataFrame()
testdf['ISBN'] = df['ISBN']
testdf['Book-Rating'] = df['Book-Rating']
testdf['User-ID'] = df['User-ID']
testdf = testdf[['User-ID','Book-Rating']].groupby(testdf['ISBN'])
print(testdf.first())

          User-ID  Book-Rating
ISBN                          
12345670   276725            9
12345671   276746            6
12345672   276745            7
12345673   276744            8
12345675   276736            9
12345677   276729            8
12345678   276727            6
12345679   276725            9


In [10]:
testdf = pd.DataFrame()
testdf['ISBN'] = df['ISBN']
testdf['Book-Rating'] = df['Book-Rating']
testdf['User-ID'] = df['User-ID']
testdf = testdf[['User-ID','Book-Rating']].groupby(testdf['ISBN'])
print(testdf.groups)

{12345670: [1], 12345671: [10], 12345672: [9], 12345673: [8], 12345675: [7], 12345677: [6], 12345678: [4, 5], 12345679: [0, 2, 3]}


In [11]:
for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    #groupDF= groupDF.reset_index(drop = True)
    print(groupDF)
    
    #print(groupDF.iloc[0,0],groupDF.iloc[0,1])
    #Dataframe.iloc[] method is used when the index label of a data frame is something other than numeric series of 0, 1, 2, 3….n

   User-ID  Book-Rating
1   276725            9
    User-ID  Book-Rating
10   276746            6
   User-ID  Book-Rating
9   276745            7
   User-ID  Book-Rating
8   276744            8
   User-ID  Book-Rating
7   276736            9
   User-ID  Book-Rating
6   276729            8
   User-ID  Book-Rating
4   276727            6
5   276737            9
   User-ID  Book-Rating
0   276725            9
2   276726            7
3   276733           10


In [36]:
for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    #print(testdf.get_group(groupKey))
    groupDF= groupDF.reset_index(drop = True)
    print(groupDF)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]] = groupDF.iloc[i,1]
        #print(i)
    print(tempDict)

   User-ID  Book-Rating
0   276725            9
{276725: 9}
   User-ID  Book-Rating
0   276746            6
{276746: 6}
   User-ID  Book-Rating
0   276745            7
{276745: 7}
   User-ID  Book-Rating
0   276744            8
{276744: 8}
   User-ID  Book-Rating
0   276736            9
{276736: 9}
   User-ID  Book-Rating
0   276729            8
{276729: 8}
   User-ID  Book-Rating
0   276727            6
1   276737            9
{276727: 6, 276737: 9}
   User-ID  Book-Rating
0   276725            9
1   276726            7
2   276733           10
{276725: 9, 276726: 7, 276733: 10}


In [ ]:
for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    print(testdf.get_group(groupKey))
    groupDF= groupDF.reset_index(drop = True)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]] = groupDF.iloc[i,1]
        #print(i)
    print(tempDict)

In [13]:
listOfDictonaries=[]
indexMap = {}
ptr=0

for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    groupDF= groupDF.reset_index(drop = True)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]] = groupDF.iloc[i,1] #[i,0]=user-id [i,1]=rating
    #in tempdict key=userid value=rating
    indexMap[ptr]=groupKey #indexMap has isbn values
    ptr=ptr+1
    listOfDictonaries.append(tempDict) 
print(indexMap)

{0: 12345670, 1: 12345671, 2: 12345672, 3: 12345673, 4: 12345675, 5: 12345677, 6: 12345678, 7: 12345679}


In [14]:
print(listOfDictonaries)

[{276725: 9}, {276746: 6}, {276745: 7}, {276744: 8}, {276736: 9}, {276729: 8}, {276727: 6, 276737: 9}, {276725: 9, 276726: 7, 276733: 10}]


In [44]:
 #This transformer turns lists of mappings (dict-like objects) 
#of feature names to feature values into arrays
uimatrix= DictVectorizer(sparse=False).fit_transform(listOfDictonaries)
#print(dictVectorizer.inverse_transform(uimatrix))
print(uimatrix)
#pairwiseSimilarity = cosine_similarity(uimatrix)

[[ 9.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  6.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  7.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  8.  0.  0.]
 [ 0.  0.  0.  0.  0.  9.  0.  0.  0.  0.]
 [ 0.  0.  0.  8.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  6.  0.  0.  0.  9.  0.  0.  0.]
 [ 9.  7.  0.  0. 10.  0.  0.  0.  0.  0.]]


In [28]:
pairwiseSimilarity = cosine_similarity(uimatrix)
print(pairwiseSimilarity)

[[1.         0.         0.         0.         0.         0.
  0.         0.59344243]
 [0.         1.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         1.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         1.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  1.         0.        ]
 [0.59344243 0.         0.         0.         0.         0.
  0.         1.        ]]


In [29]:
df = pd.DataFrame(dataset1['Book-Title'].value_counts())
df['Total-Ratings'] = df['Book-Title']
df['Book-Title'] = df.index
df.reset_index(level=0, inplace=True)
df = df.drop('index',axis=1)
df = dataset1.merge(df, left_on = 'Book-Title', right_on = 'Book-Title', how = 'left')

listOfDictonaries=[]
reverseIndexMap = {}
indexMap = {}
ptr=0

for groupKey in testdf.groups.keys():
    tempDict={}
    groupDF = testdf.get_group(groupKey)
    groupDF= groupDF.reset_index(drop = True)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]] = groupDF.iloc[i,1] #[i,0]=user-id [i,1]=rating
    #in tempdict key=userid value=rating
    indexMap[ptr]=groupKey #indexMap has isbn values
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict) 

#print(indexMap)
dictVectorizer = DictVectorizer(sparse=True)
uimatrix = dictVectorizer.fit_transform(listOfDictonaries)
#print(uimatrix)
pairwiseSimilarity = cosine_similarity(uimatrix)
print(indexMap)
print(reverseIndexMap)

{0: 12345670, 1: 12345671, 2: 12345672, 3: 12345673, 4: 12345675, 5: 12345677, 6: 12345678, 7: 12345679}
{12345670: 0, 12345671: 1, 12345672: 2, 12345673: 3, 12345675: 4, 12345677: 5, 12345678: 6, 12345679: 7}


In [51]:
    
def getTopRecommandations(bookID):
    collaborative = []
    row = reverseIndexMap[bookID]
    print("\nRECOMMENDATIONS:\n")
    
    mn = 0
    similar = []
    print(row)
    print(np.argsort(pairwiseSimilarity[row])[:-1][::-1])
    for i in np.argsort(pairwiseSimilarity[row])[:-1][::-1]:
          if dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0] not in similar:
                #print(dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0])
                #print(dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values)
                if mn>=1:
                      break
                mn+=1
                similar.append(dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0])
                collaborative.append(dataset1[dataset1['ISBN']==indexMap[i]]['Book-Title'].values[0])
    return collaborative

In [53]:
bookName=input("Enter Book Name: ")
k = list(dataset1['Book-Title'])
m = list(dataset1['ISBN'])
print(k)
print(m)
print(m[k.index(bookName)])
collaborative = getTopRecommandations(m[k.index(bookName)])
print(collaborative)

Enter Book Name: ABC
['ABC', 'OPQ', 'ABC', 'ABC', 'BCD', 'BCD', 'DEF', 'GHI', 'JKL', 'KLM', 'MNO']
[12345679, 12345670, 12345679, 12345679, 12345678, 12345678, 12345677, 12345675, 12345673, 12345672, 12345671]
12345679

RECOMMENDATIONS:

7
[0 6 5 4 3 2 1]
['OPQ']


In [ ]:
#15.165750888103101108513650872564